In [58]:
import pandas as pd
import re
import math
import time
import datetime
import numpy as np
import nltk
import gensim
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.util import bigrams
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk import everygrams
from gensim.models import Word2Vec
from gensim import models
from gensim import corpora
from gensim import similarities
from openai.embeddings_utils import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity

## Unigram model

In [52]:
df = pd.read_excel('amazon_review_processed_full.xlsx')
#df.columns

In [53]:
# Tokenisation (Full Review contains processed text - applied stemming and stopword removal, spelling check)
df['Tokenized Full review'] = df['Full review'].apply(lambda x: word_tokenize(str(x)) if isinstance(x, str) else [])
#df[['Full review', 'Tokenized Full review']].head(5)

### Raw term frequency

In [42]:
# Create a dictionary from the tokenized content, bag of words and reverse index
dictionary = corpora.Dictionary(df['Tokenized Full review'])
corpus = [dictionary.doc2bow(text) for text in df['Tokenized Full review']]
Index = similarities.SparseMatrixSimilarity(corpus, len(dictionary))

In [43]:
def query_raw(text):
    qList = text.split()  
    qLower = [w.lower() for w in qList]
    stemmer = PorterStemmer()
    qStemmed = [stemmer.stem(w) for w in qLower]
    qVector = dictionary.doc2bow(qStemmed)
    return qVector

In [44]:
qVector = query_raw('low quality')
simRaw = Index[qVector]
df['Similarity_Raw'] = simRaw
df_raw = df.sort_values(by = 'Similarity_Raw', ascending=False)
selected_columns = ["Review Model", "Review date", "Review rating", "Original title", "Original review", "Similarity_Raw"]
pd.set_option('display.max_colwidth', None)
df_raw[selected_columns].head(5)

,Review Model,Review date,Review rating,Original title,Original review,Similarity_Raw
686,Canon PIXMA TS3520,2022-11-13,5,good quality low price,Very pleased with printer. Good quality for a low price.,0.685994
5238,Canon PIXMA TR4720,2023-08-16,2,Low quality printing,"It was easy to install, but the printing job sucked. I decided to use it to print PP slides for the first time and the quality was very low.",0.685994
3805,HP DeskJet 2755e,2023-04-29,4,Quality?,Print quality not what I would expect from HP.,0.577350
9777,Canon PIXMA TR4720,2023-07-25,3,"Loud and not high quality, but it’s cheap","Beware the quality is very low on this product, but it does the job that I need. It’s not the quietest printer I’ve had. You get what you pay for.",0.547723
7068,Epson - ET-4850,2023-05-14,1,Very Poor Quality,I have had this printer for several months. It is cheap quality. Paper constantly jams. The interface is baffling and nearly unusable. Now it jams every time I use it. Just a very low quality piece of equipment.,0.544331


In [45]:
qVector = query_raw('Paper jam')
simRaw = Index[qVector]
df['Similarity_Raw'] = simRaw
df_raw = df.sort_values(by = 'Similarity_Raw', ascending=False)
selected_columns = ["Review Model", "Review date", "Review rating", "Original title", "Original review", "Similarity_Raw"]
pd.set_option('display.max_colwidth', None)
df_raw[selected_columns].head(5)

,Review Model,Review date,Review rating,Original title,Original review,Similarity_Raw
2374,Canon PIXMA TR4720,2022-12-01,2,Printer Jam,"The printer jams each time I print, very annoying. Ink runs out quickly. I barely use the printer due to paper jamming.",0.547723
3944,HP ENVY 6455e,2022-01-22,4,Paper tends to jam,Paper tends to jam,0.500000
2470,HP Smart Tank 7301,2023-01-28,1,Jams constantly,The printer consistently jams. It wont print anything. It wont even print a page without jamming.,0.486664
8904,HP ENVY Inspire 7955e,2023-08-21,1,paper jams CONSTANTLY,"Every time I try to print more than one page - the pages get jammed (although no paper actually gets stuck - it partially prints and then says jammed). I've tried different papers - thinner, thicker - it doesn't matter. It prints 2 pages and then jams - prints 2 more and jams. It partially prints and says it's jammed and ended up wasting over 30 pieces of paper trying to print an 11 page document.",0.474100
400,HP OfficeJet Pro 9015e,2022-03-22,3,continues to have paper jams,"what can I do, this printer continues to have paper jams and is not working",0.471405


### TF-IDF

In [46]:
# Create a TFIDF reverse index
TFIDF = models.TfidfModel(corpus)
corpus_TFIDF = [TFIDF[vec] for vec in corpus]
IndexTFIDF = similarities.SparseMatrixSimilarity(corpus_TFIDF, len(dictionary))

In [47]:
def query_tfidf(text):
    qList = text.split()  
    qLower = [w.lower() for w in qList]
    stemmer = PorterStemmer()
    qStemmed = [stemmer.stem(w) for w in qLower]
    qVector = dictionary.doc2bow(qStemmed)
    qVectorTFIDF = TFIDF[qVector]
    return qVectorTFIDF

In [54]:
qVectorTFIDF = query_tfidf('low quality')
simTFIDF = IndexTFIDF[qVectorTFIDF]
df['Similarity_TFIDF'] = simTFIDF
df_tfidf = df.sort_values(by = 'Similarity_TFIDF', ascending=False)
selected_columns = ["Review Model", "Review date", "Review rating", "Original title", "Original review", "Similarity_TFIDF"]
pd.set_option('display.max_colwidth', None)
df_tfidf[selected_columns].head(5)

,Review Model,Review date,Review rating,Original title,Original review,Similarity_TFIDF
686,Canon PIXMA TS3520,2022-11-13,5,good quality low price,Very pleased with printer. Good quality for a low price.,0.751272
5238,Canon PIXMA TR4720,2023-08-16,2,Low quality printing,"It was easy to install, but the printing job sucked. I decided to use it to print PP slides for the first time and the quality was very low.",0.597942
3187,HP DeskJet 2755e,2023-08-17,5,low price,setup took time,0.558956
1593,HP ENVY 6055e,2021-12-03,3,Low Quality,this printer disconnects all the time. I spend a lot of time trying to print. the printer came with toner but my printer is reading as it is low ink?? it's really frustrating trying to use this.,0.540725
1627,HP Smart Tank 5101,2023-09-10,5,It's works,This one is worth it it has where I can see the when the ink gets low,0.512525


In [55]:
qVectorTFIDF = query_tfidf('paper jam')
simTFIDF = IndexTFIDF[qVectorTFIDF]
df['Similarity_TFIDF'] = simTFIDF
df_tfidf = df.sort_values(by = 'Similarity_TFIDF', ascending=False)
selected_columns = ["Review Model", "Review date", "Review rating", "Original title", "Original review", "Similarity_TFIDF"]
pd.set_option('display.max_colwidth', None)
df_tfidf[selected_columns].head(5)

,Review Model,Review date,Review rating,Original title,Original review,Similarity_TFIDF
6611,Epson - ET-4850,2023-08-16,1,Paper Jam ALL THE TIME,"I need a copier that I can feed at least a few papers through at a time but this one doesn't copy even one paper without a paper jam, constantly, over and over again. If you need a copier that will consistently work, this one is not it.",0.454711
6179,Canon Pixma TS6420a,2022-08-10,4,Works fine,"I switched from HP to Canon. This printer prints well and offers affordable subscription plans. It is very easy to set up if you follow the instructions in the box and on the screen. I like the fact that it has wireless connection. The printer prints fast but 'moves'. The quality of the photos is excellent. It prints well on papers as well, but depending on the picture quality or paper quality, it sometimes prints papers as bluish with gray. As a student, I use it mostly for printing research papers.",0.443145
8904,HP ENVY Inspire 7955e,2023-08-21,1,paper jams CONSTANTLY,"Every time I try to print more than one page - the pages get jammed (although no paper actually gets stuck - it partially prints and then says jammed). I've tried different papers - thinner, thicker - it doesn't matter. It prints 2 pages and then jams - prints 2 more and jams. It partially prints and says it's jammed and ended up wasting over 30 pieces of paper trying to print an 11 page document.",0.424191
1473,Canon PIXMA TR4720,2023-03-05,1,Jams and Doesn't Print Correctly,"This printer is awful. Just after the return window was closed, it stopped working! It tries to suck up too many papers at once and jams every single time I print more than one sheet. I had high hopes for this printer, but it was just a waste of money. I'm super disappointed! I have even tried different brands of paper but it still jams. It prints blurry, but I'm not sure if that's from being a crappy printer or from wrestling jammed paper out of it all the time. Either way, avoid this one!!",0.405851
1554,Epson - Workforce 3820,2023-07-12,3,Pick Roller ineffective,"This printer has a pick roller issue. It will often get papers jam, picking multiple papers at once and it is a bit slower compared to similar printers.",0.397878


## Bigram model

In [79]:
#preprocess data to extract bigrams from original reviews 
df['Original_full'] = df['Original title']+df['Original review']
#df['Original_full'].head(5)

In [91]:
# Define a function to preprocess and tokenize the text
def preprocess_text(text):
    if isinstance(text, str):
        # Lowercase the text
        text = text.lower()

        # Remove non-alphanumeric characters
        text = re.sub(r'[^\w\s]', ' ', text)

        # Tokenize the text
        unigrams = word_tokenize(text)
        bigrams_list = list(bigrams(unigrams))

        # Stemming
        stemmer = PorterStemmer()
        stemmed_unigrams = [stemmer.stem(word) for word in unigrams]
        stemmed_bigrams = [tuple(stemmer.stem(word) for word in bigram) for bigram in bigrams_list]

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        unigrams_without_stopwords = [word for word in stemmed_unigrams if word not in stop_words]
        bigrams_without_stopwords = [' '.join(bigram) for bigram in stemmed_bigrams if not any(word in stop_words for word in bigram)]

        # Join both unigrams and bigrams
        tokens = unigrams_without_stopwords + bigrams_without_stopwords
    else:
        tokens = [] 

    return tokens

# Apply the modified preprocessing function to review data
df['Processed_bigram'] = df['Original_full'].apply(preprocess_text)

In [93]:
#df['Processed_bigram'].head(5)

### Raw Term Frequency

In [121]:
dictionary_bi = corpora.Dictionary(df['Processed_bigram'])
corpus_bi = [dictionary_bi.doc2bow(text) for text in df['Processed_bigram']]
Index_bi = similarities.SparseMatrixSimilarity(corpus_bi, len(dictionary_bi))

df['Vector_Raw_Bigram'] = corpus_bi

In [99]:
def query_raw_bi(query):
    preprocessed_query = preprocess_text(query)
    qVector_bi = dictionary_bi.doc2bow(preprocessed_query)
    return qVector_bi

In [101]:
qVector_bi = query_raw_bi('low quality')
SimRaw_bi = Index_bi[qVector_bi]
df['Similarity_Raw_Bigram'] = SimRaw_bi
df_raw_bi = df.sort_values(by = 'Similarity_Raw_Bigram', ascending=False)
selected_columns = ["Review Model", "Review date", "Review rating", "Original title", "Original review", "Similarity_Raw_Bigram"]
pd.set_option('display.max_colwidth', None)
df_raw_bi[selected_columns].head(5)

,Review Model,Review date,Review rating,Original title,Original review,Similarity_Raw_Bigram
686,Canon PIXMA TS3520,2022-11-13,5,good quality low price,Very pleased with printer. Good quality for a low price.,0.461880
5238,Canon PIXMA TR4720,2023-08-16,2,Low quality printing,"It was easy to install, but the printing job sucked. I decided to use it to print PP slides for the first time and the quality was very low.",0.456435
9777,Canon PIXMA TR4720,2023-07-25,3,"Loud and not high quality, but it’s cheap","Beware the quality is very low on this product, but it does the job that I need. It’s not the quietest printer I’ve had. You get what you pay for.",0.369274
3805,HP DeskJet 2755e,2023-04-29,4,Quality?,Print quality not what I would expect from HP.,0.348155
7068,Epson - ET-4850,2023-05-14,1,Very Poor Quality,I have had this printer for several months. It is cheap quality. Paper constantly jams. The interface is baffling and nearly unusable. Now it jams every time I use it. Just a very low quality piece of equipment.,0.344265


In [100]:
qVector_bi = query_raw_bi('Paper jam')
SimRaw_bi = Index_bi[qVector_bi]
df['Similarity_Raw_Bigram'] = SimRaw_bi
df_raw_bi = df.sort_values(by = 'Similarity_Raw_Bigram', ascending=False)
selected_columns = ["Review Model", "Review date", "Review rating", "Original title", "Original review", "Similarity_Raw_Bigram"]
pd.set_option('display.max_colwidth', None)
df_raw_bi[selected_columns].head(5)

,Review Model,Review date,Review rating,Original title,Original review,Similarity_Raw_Bigram
5691,HP OfficeJet Pro 8025e,2021-10-17,2,Poor quality.,"If you print something? Paper jams. Reload the paper the only way it can go? Paper jam.WiFi blinks? Paper jam. You leave the printer alone too long? Paper jam. It’s incredible how hard it must be to make a printer that just prints when it’s told to, because no one can seem to do it. I’ve had this for 3 months and it’s a very high maintenance printer.",0.712396
9937,HP Smart Tank 6001,2023-07-19,3,Paper jam issues,Always gives a paper jam error when trying to print when there is no jam. Unplug wait a little then plug back in and works fine,0.673575
4537,Epson - ET-4850,2022-12-22,2,multiple paper jams every day,multiple paper jams every day.,0.666667
4545,Canon PIXMA MG3620,2023-06-28,1,Consistent paper jams,"Most of the paper jams I could fix. However, I had a paper jam so bad, it destroyed the rollers so I used the warranty. Then, it happened again. Do yourself a favor, buy a different printer.",0.655610
5417,Canon Pixma TS6420a,2022-07-02,3,Paper Jams easily,It has an easy set up but every time I prin the paper jams. Still better than an HP,0.654654


In [114]:
qVector_bi = query_raw_bi('Customer support')
SimRaw_bi = Index_bi[qVector_bi]
df['Similarity_Raw_Bigram'] = SimRaw_bi
df_raw_bi = df.sort_values(by = 'Similarity_Raw_Bigram', ascending=False)
selected_columns = ["Review Model", "Review date", "Review rating", "Original title", "Original review", "Similarity_Raw_Bigram"]
pd.set_option('display.max_colwidth', None)
df_raw_bi[selected_columns].head(5)

,Review Model,Review date,Review rating,Original title,Original review,Similarity_Raw_Bigram
6889,HP OfficeJet Pro 8034e,2023-06-12,1,Worst Customer Support Ever!,I just loaded brand new cartridges and my printing is completely faded. Absolutely horrible getting in touch with customer support to help. Do not buy!,0.577350
3330,Canon PIXMA MG3620,2023-07-26,3,Powers completely off when not in use,"Returning this. For 1. when not in use it shuts down completely. I don’t know how long it stays idle before shutting down and there’s no way to find out because it’s not in the manual nor can you call customer support without having an account and providing your email and phone number. I also tried to call customer support to get an answer of is a Google Chromebook compatible? It does not appear to be but that is not listed anywhere in the instructions and again, I can’t speak to anyone in customer support unless I create an account. It took me awhile to set it up wirelessly but I succeeded and have printed from my iPhone a few times, but the Chromebook is a no go.",0.504695
8300,HP DeskJet 2755e,2023-08-08,3,Help printer woes,Had to call customer support to set up/ link to my devices,0.480384
8753,Canon PIXMA TS3520,2022-03-11,1,RETURN,"First one wouldn't load ink, second one will not connect to the internet. Tried to contact customer support, ha, ha. What support",0.421637
9864,Canon PIXMA TR8620a,2023-01-26,5,great printer,"As someone who worked in the office supply business and dealt with customer support for 12 years, I knew I wanted a Canon. We called other brands disposable due to a lack of repair support. I knew Canon was a brand that if I had any issues with, their customer support (although can be difficult to get hold of) will take care of the issue. So far, the printer was easily set up, works great, and has great print quality.",0.406181


### TF-IDF

In [120]:
# Create a TFIDF reverse index
TFIDF_bi = models.TfidfModel(corpus_bi)
corpus_TFIDF_bi = [TFIDF_bi[vec] for vec in corpus_bi]
IndexTFIDF_bi = similarities.SparseMatrixSimilarity(corpus_TFIDF_bi, len(dictionary_bi))

df['Vector_TFIDF_Bigram'] = corpus_TFIDF_bi

In [116]:
def query_tfidf_bi(query):
    preprocessed_query = preprocess_text(query)
    qVector_bi = dictionary_bi.doc2bow(preprocessed_query)
    qVectorTFIDF_bi = TFIDF_bi[qVector_bi]
    return qVectorTFIDF_bi

In [117]:
qVectorTFIDF_bi = query_tfidf_bi('low quality')
simTFIDF_bi = IndexTFIDF_bi[qVectorTFIDF_bi]
df['Similarity_TFIDF_Bigram'] = simTFIDF_bi
df_tfidf_bi = df.sort_values(by = 'Similarity_TFIDF_Bigram', ascending=False)
selected_columns = ["Review Model", "Review date", "Review rating", "Original title", "Original review", "Similarity_TFIDF_Bigram"]
pd.set_option('display.max_colwidth', None)
df_tfidf_bi[selected_columns].head(5)

,Review Model,Review date,Review rating,Original title,Original review,Similarity_TFIDF_Bigram
5238,Canon PIXMA TR4720,2023-08-16,2,Low quality printing,"It was easy to install, but the printing job sucked. I decided to use it to print PP slides for the first time and the quality was very low.",0.318081
7068,Epson - ET-4850,2023-05-14,1,Very Poor Quality,I have had this printer for several months. It is cheap quality. Paper constantly jams. The interface is baffling and nearly unusable. Now it jams every time I use it. Just a very low quality piece of equipment.,0.251081
3767,Canon PIXMA TR4720,2023-06-07,1,Item had obviously been used. Ink cartridges were dried up. Very disappointing !,"Printer did not produce quality copies. It arrived with original packaging already removed and Ink installed and dry. With new ink, the copies were low quality. Very disappointed in Cannon.",0.240806
4819,Epson - XP-6100,2022-10-26,3,Could be better,Just taking it out of the box the printers plastic felt cheap and fragile. Print quality seems a little low quality. The touch screen is better than past models but it's placement again is very awkward. The mechanism it sits on seems very easy to break.,0.221226
6853,HP ENVY Inspire 7255e,2023-03-29,3,Quality is poor unless you print a lot,"I learned that you have to print often or else the ink will clog and produce low quality documents. I learned how to fix the issue from a video, but what a pain. (You soak the ink cartridge bottom in warm water for five minutes and then wipe it off with a towel.) I've had HP printers (and other brands) in the past and have never had this issue.",0.211376


In [118]:
qVectorTFIDF_bi = query_tfidf_bi('paper jam')
simTFIDF_bi = IndexTFIDF_bi[qVectorTFIDF_bi]
df['Similarity_TFIDF_Bigram'] = simTFIDF_bi
df_tfidf_bi = df.sort_values(by = 'Similarity_TFIDF_Bigram', ascending=False)
selected_columns = ["Review Model", "Review date", "Review rating", "Original title", "Original review", "Similarity_TFIDF_Bigram"]
pd.set_option('display.max_colwidth', None)
df_tfidf_bi[selected_columns].head(5)

,Review Model,Review date,Review rating,Original title,Original review,Similarity_TFIDF_Bigram
3766,Epson - ET-2850,2023-06-13,1,Did not work.,"Printer did not work. It had a paper jam when I was setting up the printer. I cleared the paper jam as directed but the printer would not clear the paper jam error on the printer screen. I called Epson for help, and they said to just send it back.",0.627548
5691,HP OfficeJet Pro 8025e,2021-10-17,2,Poor quality.,"If you print something? Paper jams. Reload the paper the only way it can go? Paper jam.WiFi blinks? Paper jam. You leave the printer alone too long? Paper jam. It’s incredible how hard it must be to make a printer that just prints when it’s told to, because no one can seem to do it. I’ve had this for 3 months and it’s a very high maintenance printer.",0.503509
5398,HP ENVY 6455e,2021-10-05,2,Printer jams …. All the time!,It is difficult to connect to the internet and the paper jams constantly,0.501721
5998,HP ENVY 6055e,2021-09-28,4,Easy to use but paper jams and photo quality lacking.,"I was expecting high quality photo prints. It does the job of a printer but not to the quality I'd like. Easy to use but does ""paper jam"" constantly despite there not being a paper jam. 🤔",0.469767
3130,HP Smart Tank 6001,2022-12-25,2,Kept on getting paper jam had to return,Since day one I kept on getting paper jams and then one day a glitch where there's no paper but it's jammed did everything I could to fix it in the end just returned it,0.447230


In [119]:
qVectorTFIDF_bi = query_tfidf_bi('customer support')
simTFIDF_bi = IndexTFIDF_bi[qVectorTFIDF_bi]
df['Similarity_TFIDF_Bigram'] = simTFIDF_bi
df_tfidf_bi = df.sort_values(by = 'Similarity_TFIDF_Bigram', ascending=False)
selected_columns = ["Review Model", "Review date", "Review rating", "Original title", "Original review", "Similarity_TFIDF_Bigram"]
pd.set_option('display.max_colwidth', None)
df_tfidf_bi[selected_columns].head(5)

,Review Model,Review date,Review rating,Original title,Original review,Similarity_TFIDF_Bigram
6889,HP OfficeJet Pro 8034e,2023-06-12,1,Worst Customer Support Ever!,I just loaded brand new cartridges and my printing is completely faded. Absolutely horrible getting in touch with customer support to help. Do not buy!,0.392759
3330,Canon PIXMA MG3620,2023-07-26,3,Powers completely off when not in use,"Returning this. For 1. when not in use it shuts down completely. I don’t know how long it stays idle before shutting down and there’s no way to find out because it’s not in the manual nor can you call customer support without having an account and providing your email and phone number. I also tried to call customer support to get an answer of is a Google Chromebook compatible? It does not appear to be but that is not listed anywhere in the instructions and again, I can’t speak to anyone in customer support unless I create an account. It took me awhile to set it up wirelessly but I succeeded and have printed from my iPhone a few times, but the Chromebook is a no go.",0.360282
6289,Epson - Workforce 4820,2020-10-21,1,Software issues - do not order this printer,"I received my first printer and kept getting an error saying ""insert paper cassette correctly"". I am no novice to printers but I called customer support because no matter what I did it wasn't working. Customer support determined it was a software problem and to replace the unit. I did so and received my new one. Yayyy, I finally have a printer after months of back ordering and running to UPS for my print jobs. Nope, this one had the exact same issue, and I was told the exact same thing by customer support.Apparently this is a new model that was just released and they haven't worked out the bugs yet. DO NOT PURCHASE THIS PRINTER.",0.334134
8300,HP DeskJet 2755e,2023-08-08,3,Help printer woes,Had to call customer support to set up/ link to my devices,0.301210
9864,Canon PIXMA TR8620a,2023-01-26,5,great printer,"As someone who worked in the office supply business and dealt with customer support for 12 years, I knew I wanted a Canon. We called other brands disposable due to a lack of repair support. I knew Canon was a brand that if I had any issues with, their customer support (although can be difficult to get hold of) will take care of the issue. So far, the printer was easily set up, works great, and has great print quality.",0.276070


## Export Data

In [122]:
#Export to csv file to run on Streamlit
columns = ['Review Model', 'Review date', 'Review name',
       'Review rating', 'Full review',
       'Verified Purchase or not', 'People_find_helpful', 'vine or not',
       'list price', 'rating count', 'overall rating', 'Original title',
       'Original review',  'Brand',
        'Vector_Raw_Bigram', 'Vector_TFIDF_Bigram'
      ]
df_final = df[columns]
df_final.to_csv('document_retrieval.csv') 

# Use tensorflow to embed

In [11]:
#Use tensorflow to embed
# df['Review Content new'] = df['Review Content'].apply(lambda x: str(x) if x is not None else '') 

# def clean_text(text):
#     text = re.sub(r'[^a-zA-Z\s]', '', text)
#     text = text.lower()
#     return text

# df['Cleaned Reviews'] = df['Review Content new'].apply(clean_text)

# def remove_stopwords(text):
#     stop_words = set(stopwords.words('english'))
#     words = text.split()
#     filtered_words = [word for word in words if word.lower() not in stop_words]
#     return ' '.join(filtered_words)

# df['Final Reviews'] = df['Cleaned Reviews'].apply(remove_stopwords)

# model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# def embed(text,model):
#     embeddings = model(text)
#     return [embedding.numpy() for embedding in embeddings]


# df['Embed_sentence'] = embed(df['Final Reviews'], model)

# df['Embed_sentence']

In [20]:
def embed_label(text,model):
    embeddings = model([text])
    return embeddings.numpy()[0]

text = 'Connectivity'
input_embedding_vector = embed_label(text,model)

df['similarity'] = df['Embed_sentence'].apply(lambda x: cosine_similarity(input_embedding_vector,x))

In [22]:
pd.set_option('display.max_colwidth', None)
df_new = df.sort_values(by = 'similarity',ascending = False)
columns = ['Brand','Review Model','Review rating','Review Content']
df_select = df_new[columns]
df_final = df_select[df_select['Brand'] == 'HP'].head(4).reset_index(drop = True)
df_final

,Brand,Review Model,Review rating,Review Content
0,HP,HP Smart Tank 6001,1,Connection problems
1,HP,HP ENVY Inspire 7955e,5,So easy to connect
2,HP,HP DeskJet 2755e,5,Connected to devices with no problem.
3,HP,HP Smart Tank 7301,2,wireless will not stay connected


In [25]:
text = 'Print Quality'
input_embedding_vector = embed_label(text,model)

df['similarity'] = df['Embed_sentence'].apply(lambda x: cosine_similarity(input_embedding_vector,x))

pd.set_option('display.max_colwidth', None)
df_new = df.sort_values(by = 'similarity',ascending = False)
columns = ['Brand','Review Model','Review rating','Review Content']
df_select = df_new[columns]
df_final = df_select[df_select['Brand'] == 'HP'].head(4).reset_index(drop = True)
df_final

,Brand,Review Model,Review rating,Review Content
0,HP,HP ENVY Inspire 7955e,5,I have been pleased with the print quality
1,HP,HP OfficeJet Pro 9025e,5,Print quality exactly. Best one I have had.
2,HP,HP ENVY 6055e,5,The print quality is good. I am very pleased.
3,HP,HP ENVY 6455e,5,Excellent print quality !!


In [34]:
text = 'Paper jam'
input_embedding_vector = embed_label(text,model)

df['similarity'] = df['Embed_sentence'].apply(lambda x: cosine_similarity(input_embedding_vector,x))

pd.set_option('display.max_colwidth', None)
df_new = df.sort_values(by = 'similarity',ascending = False)
columns = ['Brand','Review Model','Review rating','Review Content']
df_select = df_new[columns]
df_final = df_select[df_select['Brand'] == 'HP'].head(4).reset_index(drop = True)
df_final

,Brand,Review Model,Review rating,Review Content
0,HP,HP ENVY 6455e,4,Paper tends to jam
1,HP,HP OfficeJet Pro 8034e,5,the paper feed in front
2,HP,HP OfficeJet Pro 9015e,3,"what can I do, this printer continues to have paper jams and is not working"
3,HP,HP OfficeJet Pro 8025e,2,"If you print something? Paper jams. Reload the paper the only way it can go? Paper jam.WiFi blinks? Paper jam. You leave the printer alone too long? Paper jam. It’s incredible how hard it must be to make a printer that just prints when it’s told to, because no one can seem to do it. I’ve had this for 3 months and it’s a very high maintenance printer."


# Other (Draft)

In [19]:
# labels = [
#     'Setup', 
#     'Connectivity', 
#     'Customer Support', 
#     'Print Quality', 
#     'Print Speed', 
#     'Ink supply and Cartridge', 
#     'Printer Hardware Robustness and sturdiness', 
#     'Control Panel', 
#     'Ease of Use', 
#     'Firmware', 
#     'Business Services and Subscription', 
#      'Paper jam',
#      'Control Panel', 
#     'Other'
# ]

# df_labels = pd.DataFrame({'Labels': labels})
# df_labels

In [ ]:
# def docs2vecs(docs,dictionary):
#     vec1 = [dictionary.doc2bow(doc) for doc in docs]
#     tfid = gensim.models.TfidfModel(vec1)
#     vec2 = [tfidf[vec] for vec in vec1]
#     return vec2

# vectorizer = TfidfVectorizer(max_df = 0.5, min_df = 2, stop_words = 'english')
# X = vectorizer.fit_transform(dataset.data)

In [85]:
# model = Word2Vec(sentences = label_token, vector_size=100, window=5, min_count=1, sg=0)

In [112]:
# from gensim.models import Word2Vec
# import pandas as pd

# # List of labels
# labels = [
#     'Setup', 
#     'Connectivity', 
#     'Customer Support', 
#     'Print Quality', 
#     'Print Speed', 
#     'Ink supply and Cartridge', 
#     'Printer Hardware Robustness and sturdiness', 
#     'Control Panel', 
#     'Ease of Use', 
#     'Firmware', 
#     'Business Services and Subscription', 
#     'Paper jam',
#     'Control Panel', 
#     'Other'
# ]

# # Create a DataFrame with labels
# df_labels = pd.DataFrame({'Labels': labels})

# # Split the labels into words (tokens)
# label_tokens = [label.split() for label in labels]

# # Train a Word2Vec model on label tokens
# model = Word2Vec(sentences=label_tokens, vector_size=100, window=5, min_count=1, sg=0)

# # Function to get the embedding of a label
# def label_embedding(label, model):
#     return model.wv[label]  # Use indexing to retrieve the embedding

# # Apply the label_embedding function to embed labels
# df_labels['Label Embeddings'] = df_labels['Labels'].apply(lambda x: label_embedding(x, model))

In [19]:
# def search_reviews(df, product_description, pprint=True):
#     product_embedding = get_embedding(
#         product_description,
#         engine=embedding_model
#     )
#     df[f"similarity_{product_description}"] = df.embedding.apply(lambda x: cosine_similarity(x, product_embedding))

#     results = df.sort_values(f"similarity_{product_description}", ascending=False)
                       
#     return results



# summary_df = []

# for label in labels:
#     label_results = search_reviews(df_filter, label, pprint=True)
    
#     # Apply a similarity threshold to filter rows
#     threshold = 0.85
#     filtered_results = label_results[label_results[f"similarity_{label}"] > threshold]
    
#     # Add the label column to the filtered results
#     filtered_results['topic'] = label
    
#     # Append the filtered results to the summary_df
#     summary_df.append(filtered_results)
    
#     time.sleep(5)

# # Concatenate the filtered results into the final summary DataFrame
# summary_df = pd.concat(summary_df)


In [30]:
# columns_to_drop = ['embedding', 'embedding_splitted']

# # Use the drop method to remove the specified columns
# df_phrase_filtered = df_phrase.drop(columns=columns_to_drop)

In [46]:
# def find_most_similar_word(review_embedding, word_vectors, word_list):
#     similarities = [cosine_similarity(review_embedding, word_vector) for word_vector in word_vectors]
#     most_similar_index = similarities.index(max(similarities))
#     return word_list[most_similar_index]  # Return the actual word

# # Assuming you have a DataFrame 'df' with 'embedding' column containing review embeddings
# df_phrase['Topic'] = df_phrase['embedding_splitted'].apply(lambda x: find_most_similar_word(x, label_embeddings, labels))


C:\Users\YeTi985\AppData\Local\Temp\ipykernel_17244\3029389664.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_phrase['Topic'] = df_phrase['embedding_splitted'].apply(lambda x: find_most_similar_word(x, label_embeddings, labels))


In [47]:
# df_phrase['Topic2'] = df_phrase['embedding'].apply(lambda x: find_most_similar_word(x, label_embeddings, labels))

C:\Users\YeTi985\AppData\Local\Temp\ipykernel_17244\3720456180.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_phrase['Topic2'] = df_phrase['embedding'].apply(lambda x: find_most_similar_word(x, label_embeddings, labels))


In [48]:
df_phrase 

,Unnamed: 0,Review Model,Retailer,Review date,Review rating,Review title,Review content,Review concat text,Tokens,key phrase,embedding,key phrase splitted,embedding_splitted,Topic,Topic2
6388,6388,HP OfficeJet Pro 9025e,Amazon,2021-05-07,3,Easy install and easy printing. Too proprietary.,Easy set up. Easy scanning. Didn’t join the i...,Retailer: Amazon - Review model: HP OfficeJet ...,198,"[Easy setup, easy scanning, Proprietary restri...","[0.00624576723203063, 0.003082588315010071, 8....",Easy setup,"[0.0028877160511910915, 0.015896936878561974, ...",Setup,Ink supply and Cartridge
6389,6389,HP OfficeJet Pro 9025e,Amazon,2021-05-07,3,Easy install and easy printing. Too proprietary.,Easy set up. Easy scanning. Didn’t join the i...,Retailer: Amazon - Review model: HP OfficeJet ...,198,"[Easy setup, easy scanning, Proprietary restri...","[0.00624576723203063, 0.003082588315010071, 8....",easy scanning,"[-0.04566308483481407, 0.025933345779776573, 0...",Ease of Use,Ink supply and Cartridge
6390,6390,HP OfficeJet Pro 9025e,Amazon,2021-05-07,3,Easy install and easy printing. Too proprietary.,Easy set up. Easy scanning. Didn’t join the i...,Retailer: Amazon - Review model: HP OfficeJet ...,198,"[Easy setup, easy scanning, Proprietary restri...","[0.00624576723203063, 0.003082588315010071, 8....",Proprietary restrictions,"[0.0037913022097200155, -0.012775414623320103,...",Firmware,Ink supply and Cartridge
6391,6391,HP OfficeJet Pro 9025e,Amazon,2021-05-07,3,Easy install and easy printing. Too proprietary.,Easy set up. Easy scanning. Didn’t join the i...,Retailer: Amazon - Review model: HP OfficeJet ...,198,"[Easy setup, easy scanning, Proprietary restri...","[0.00624576723203063, 0.003082588315010071, 8....",Inflexibility with ink brands,"[-0.0056190043687820435, 0.005538931582123041,...",Ink supply and Cartridge,Ink supply and Cartridge
6401,6401,HP OfficeJet Pro 9025e,Amazon,2023-03-03,5,Great Printer,"After years of my old printer, purchased this ...",Retailer: Amazon - Review model: HP OfficeJet ...,84,"[Fast printing, liked two tray feature, easy w...","[-0.03244659677147865, 0.009338131174445152, -...",Fast printing,"[-0.0462610200047493, 0.024372506886720657, -0...",Print Speed,Print Speed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10711,10711,HP OfficeJet Pro 9025e,Amazon,2021-05-08,5,Great for working from home,"This really is a great printer, and it's so mu...",Retailer: Amazon - Review model: HP OfficeJet ...,268,"[Excellent for home office use, works seamless...","[-0.014914722181856632, 0.0015333282062783837,...",positive response to free ink plan,"[-0.013421832583844662, -0.01247564610093832, ...",Ink supply and Cartridge,Printer Hardware Robustness and sturdiness
10712,10712,HP OfficeJet Pro 9025e,Amazon,2021-05-02,5,"Really Nice, Great Setup",So the setup process with printers is normally...,Retailer: Amazon - Review model: HP OfficeJet ...,132,"[Easy setup process, App-based setup is conven...","[-0.016588890925049782, 0.017046066001057625, ...",Easy setup process,"[0.006355913821607828, 0.021701103076338768, 0...",Setup,Ease of Use
10713,10713,HP OfficeJet Pro 9025e,Amazon,2021-05-02,5,"Really Nice, Great Setup",So the setup process with printers is normally...,Retailer: Amazon - Review model: HP OfficeJet ...,132,"[Easy setup process, App-based setup is conven...","[-0.016588890925049782, 0.017046066001057625, ...",App-based setup is convenient,"[-0.006079057231545448, 0.010124939493834972, ...",Ease of Use,Ease of Use
10714,10714,HP OfficeJet Pro 9025e,Amazon,2021-05-02,5,"Really Nice, Great Setup",So the setup process with printers is normally...,Retailer: Amazon - Review model: HP OfficeJet ...,132,"[Easy setup process, App-based setup is conven...","[-0.016588890925049782, 0.017046066001057625, ...",Supports multiple connections,"[-0.038474854081869125, 0.005594192072749138, ...",Connectivity,Ease of Use
